# IOI 因果痕迹可视化

可视化分析各层、各位置的因果痕迹分布，用于目视确认语义合理性。

生成图表：
1. **热力图**: (layer × token_pos) 痕迹范数 — 定位 IOI 回路结构
2. **PCA**: top-5 检查点痕迹的主成分分析 — 验证方向聚类
3. **余弦相似度**: 检查点之间的方向一致性 — 验证功能分组

完整实现见 `scripts/visualize_imprints.py`，本 notebook 提供交互式入口。

In [ ]:
import sys
from pathlib import Path

# 让 src/ 下的包可导入
PROJECT_ROOT = Path.cwd().parent
sys.path.insert(0, str(PROJECT_ROOT / "src"))

%matplotlib inline

In [ ]:
# 运行完整可视化脚本（也可逐步执行下方各 cell）
# !python ../scripts/visualize_imprints.py

## Step 0: 加载模型与数据

In [ ]:
from scripts.visualize_imprints import (
    prepare_data,
    plot_imprint_heatmap,
    plot_imprint_pca,
    plot_cosine_similarity,
)

model, tokenizer, clean_ids, corrupt_ids, token_labels = prepare_data(
    n_samples=100, seed=42,
)
print(f"Loaded: {clean_ids.shape[0]} samples, seq_len={clean_ids.shape[1]}")

## Plot 1: 痕迹范数热力图
预期：IOI 回路结构 — 中后层 + IO/末尾位置的范数最高

In [ ]:
norm_matrix = plot_imprint_heatmap(model, clean_ids, corrupt_ids, token_labels)

## Plot 2: PCA — Top-5 检查点痕迹的主成分分析
预期：同一检查点的样本应形成聚类（痕迹方向一致）

In [ ]:
top_imprints = plot_imprint_pca(model, clean_ids, corrupt_ids, norm_matrix, top_k=5)

## Plot 3: 余弦相似度矩阵
预期：同一功能阶段的检查点（如相邻层+相同位置）之间相似度更高

In [ ]:
plot_cosine_similarity(top_imprints)
print("All figures saved to outputs/figures/")